# **Samsung Ai (Materials Discovery)**
### Just number one

In [1]:
!pip install eli5

In [2]:
!pip install ngboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.5/349.5 kB 7.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=cc4ef221e976445fdf562c19f1f899002e74274e4aa383f061c29c1b218def15
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


In [3]:
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import random
import re
from joblib import Parallel, delayed
from glob import glob
import math
from tqdm import tqdm
import warnings
warnings.filterwarnings(action = 'ignore')

import optuna
from optuna.samplers import TPESampler

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, StackingRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb
from sklearn.inspection import permutation_importance
from ngboost import NGBRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTE
import eli5
from eli5.sklearn import PermutationImportance

import tensorflow as tf
from tensorflow.keras import layers as L
import tensorflow_addons as tfa
from tensorflow.keras import backend as K

In [5]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(SEED) # Seed 고정

In [6]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 33.9 MB/s eta 0:00:0000:0100:01


In [7]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176725 sha256=fbf35633e298d9c34fa730c52340a99f52372bcfd24d01ab38155750e66d0cfe
  Stored in directory: /root/.cache/pip/wheels/02/c0/2e/e7e3d63b431777712ebc128bc4deb9ac5cb19afc7c1ea341ec
Successfully built mordred


In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from mordred import Calculator, descriptors, Lipinski

In [9]:
samsung_folder = '../input/samsung/'
os.listdir(samsung_folder)

['sample_submission.csv', 'test_set.csv', 'train_set.ReorgE.csv', 'mol_files']

In [10]:
for folder in tqdm(os.listdir(samsung_folder)):
    if '.csv' in folder:
        print(folder)
        tmp = folder.split('.')[0]
        globals()[f'{tmp}'] = pd.read_csv(samsung_folder + f'{folder}')
    else:
        for foldersec in os.listdir(samsung_folder + 'mol_files/mol_files'):

            if os.path.isdir(f"{foldersec}_ex_file"):
                pass
            else:
                os.mkdir(f"{foldersec}_ex_file")

            if os.path.isdir(f"{foldersec}_g_file"):
                pass
            else:
                os.mkdir(f"{foldersec}_g_file")
    
            data_mol = sorted(glob(samsung_folder + f"mol_files/mol_files/{foldersec}/*.mol"))
            
            for i in data_mol:
                result = []
                tmp = open(i, 'r').read().split("\n")
                for j in tmp:
                    tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
                    if len(tmp_) > 11:
                        result.append(tmp_)

                file_name = i.split('/')[-1].split('.')[0]
                
                if "ex" in file_name:
                    pd.DataFrame(result).to_csv(f"{foldersec}_ex_file/{file_name}" + ".csv", index = False)
                elif "g" in file_name:
                    pd.DataFrame(result).to_csv(f"{foldersec}_g_file/{file_name}" + ".csv", index = False)

  0%|          | 0/4 [00:00<?, ?it/s]

sample_submission.csv
test_set.csv
train_set.ReorgE.csv


100%|██████████| 4/4 [06:21<00:00, 95.35s/it]


In [11]:
'''dropi = []
for i in range(0, train_set.shape[0]):
    if train_set['SMILES'][i] in list(test_set['SMILES']):
        dropi.append(i)

train_set.drop(dropi, axis=0, inplace = True)'''

"dropi = []\nfor i in range(0, train_set.shape[0]):\n    if train_set['SMILES'][i] in list(test_set['SMILES']):\n        dropi.append(i)\n\ntrain_set.drop(dropi, axis=0, inplace = True)"

In [12]:
#train_set = train_set[train_set.duplicated(['SMILES'], keep = 'first') == False]

In [13]:
#train_set.index = [i[1] for i in train_set['index'].str.split('_')]

#train_set = train_set.reset_index()

#train_set['level_0'] = train_set['level_0'].astype(int)

In [14]:
max_len = 0
error = []
g_data, ex_data = [], []
mol = []

for i in tqdm(train_set['index']):
    tmp_g = pd.read_csv(f"train_set_g_file/{i}" + '_g.csv')

    if len(tmp_g) >= max_len:
        max_len = len(tmp_g)
        
    for j in tmp_g["3"]:
        mol.append(j)

100%|██████████| 18157/18157 [00:33<00:00, 535.18it/s]


In [15]:
mol_list = list(set(mol))

g_data, ex_data = [], []

def get_mol(data):
    if data in mol_list:
        return mol_list.index(data) + 1
    else:
        return -1

for i in tqdm(train_set['index']):
    tmp_g = pd.read_csv(f"train_set_g_file/{i}" + '_g.csv').iloc[:, :16]
    tmp_ex = pd.read_csv(f"train_set_ex_file/{i}" +'_ex.csv').iloc[:, :16]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
    
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if max_len != len(tmp_g):
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 16] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 16] * (max_len - tmp_ex.shape[0]))))
    elif max_len == len(tmp_g):
        tmp_g = tmp_g
        tmp_ex = tmp_ex
    
    g_data.append(tmp_g)
    ex_data.append(tmp_ex)

100%|██████████| 18157/18157 [01:51<00:00, 162.49it/s]


In [16]:
test_g, test_ex = [], []

for i in tqdm(test_set[test_set.columns[0]]):
    tmp_g = pd.read_csv(f"test_set_g_file/{i}" + '_g.csv').iloc[:, :16]
    tmp_ex = pd.read_csv(f"test_set_ex_file/{i}" +'_ex.csv').iloc[:, :16]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
    
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if len(tmp_g) < max_len:
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 16] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 16] * (max_len - tmp_ex.shape[0]))))
    elif len(tmp_g) == max_len:
        pass
    else:
        tmp_g = tmp_g[:max_len]
        tmp_ex = tmp_ex[:max_len]
    
    test_g.append(tmp_g)
    test_ex.append(tmp_ex)

100%|██████████| 457/457 [00:02<00:00, 166.45it/s]


# 2D Version

In [24]:
for folder in tqdm(os.listdir(samsung_folder)):
    if '.csv' in folder:
        print(folder)
        tmp = folder.split('.')[0]
        globals()[f'{tmp}'] = pd.read_csv(samsung_folder + f'{folder}')
    else:
        pass

100%|██████████| 4/4 [00:00<00:00, 131.26it/s]

sample_submission.csv
test_set.csv
train_set.ReorgE.csv


In [28]:
dropi = []
for i in range(0, train_set.shape[0]):
    if train_set['SMILES'][i] in list(test_set['SMILES']):
        dropi.append(i)

train_set.drop(dropi, axis=0, inplace = True)

train_set.index = [i[1] for i in train_set['index'].str.split('_')]

train_set = train_set.reset_index()

train_set['level_0'] = train_set['level_0'].astype(int)

In [91]:
if os.path.isdir(f"train_set_2D_file"):
    pass
else:
    os.mkdir(f"train_set_2D_file")

for i in tqdm(train_set['level_0']):
    m = Chem.MolFromSmiles(train_set['SMILES'][train_set['level_0'] == i].values[0])
    m = Chem.MolToMolBlock(m)
    result = []
    tmp = m.split('\n')
    for j in tmp:
        tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
        if len(tmp_) > 11:
            result.append(tmp_)

    file_name = 'train_' + str(i)

    if file_name:
        pd.DataFrame(result).to_csv(f"train_set_2D_file/{file_name}" + ".csv", index = False)

100%|██████████| 18155/18155 [00:49<00:00, 365.38it/s]


In [97]:
max_len = 0
error = []
mol = []

for i in tqdm(train_set['index']):
    tmp_g = pd.read_csv(f"train_set_2D_file/{i}" + '.csv')

    if len(tmp_g) >= max_len:
        max_len = len(tmp_g)
        
    for j in tmp_g["3"]:
        mol.append(j)

100%|██████████| 18155/18155 [00:33<00:00, 541.79it/s]


In [98]:
mol_list = list(set(mol))

train2D = []

def get_mol(data):
    if data in mol_list:
        return mol_list.index(data) + 1
    
    else:
        return -1

for i in tqdm(train_set['index']):
    
    tmp = pd.read_csv(f"train_set_2D_file/{i}" + '.csv').iloc[:, :16]
    
    tmp["3"] = tmp["3"].apply(lambda x : get_mol(x))
    
    tmp = np.array(tmp)
    
    if max_len != len(tmp):
        tmp = np.concatenate((tmp, np.array([[0] * 16] * (max_len-tmp.shape[0]))))
    elif max_len == len(tmp):
        tmp = tmp
    
    train2D.append(tmp)

100%|██████████| 18155/18155 [00:52<00:00, 344.82it/s]


In [99]:
#train2D = np.array(train2D).reshape(len(train2D), -1)

#test_x = train2D

#print(test_x.shape)

train2D_ = np.array(train2D)

In [100]:
if os.path.isdir(f"test_set_2D_file"):
    pass
else:
    os.mkdir(f"test_set_2D_file")

for i in tqdm(range(0, test_set.shape[0])):
    m = Chem.MolFromSmiles(test_set['SMILES'][i])
    m = Chem.MolToMolBlock(m)
    result = []
    tmp = m.split('\n')
    for j in tmp:
        tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
        if len(tmp_) > 11:
            result.append(tmp_)
    
    file_name = 'test_' + str(i)

    if file_name:
        pd.DataFrame(result).to_csv(f"test_set_2D_file/{file_name}" + ".csv", index = False)

100%|██████████| 457/457 [00:01<00:00, 317.24it/s]


In [101]:
test2D = []

for i in tqdm(test_set[test_set.columns[0]]):
    tmp = pd.read_csv(f"test_set_2D_file/{i}" + '.csv').iloc[:, :16]
    
    tmp["3"] = tmp["3"].apply(lambda x : get_mol(x))
    
    tmp = np.array(tmp)
    
    if len(tmp) < max_len:
        tmp = np.concatenate((tmp, np.array([[0] * 16] * (max_len-tmp.shape[0]))))
    elif len(tmp) == max_len:
        pass
    else:
        tmp = tmp[:max_len]
    
    test2D.append(tmp)

100%|██████████| 457/457 [00:01<00:00, 302.91it/s]


In [102]:
#test2D = np.array(test2D).reshape(len(test2D), -1)

#test_x = test2D

#print(test_x.shape)

test2D_ = np.array(test2D)

# 분자 특성

In [116]:
from rdkit.Chem import Descriptors

In [117]:
train_set['mol'] = train_set['SMILES'].apply(lambda x: Chem.MolFromSmiles(x)) 
train_set['mol'] = train_set['mol'].apply(lambda x: Chem.AddHs(x))
train_set['num_of_atoms'] = train_set['mol'].apply(lambda x: x.GetNumAtoms())
train_set['num_of_heavy_atoms'] = train_set['mol'].apply(lambda x: x.GetNumHeavyAtoms())

train_set['tpsa'] = train_set['mol'].apply(lambda x: Descriptors.TPSA(x))
train_set['mol_w'] = train_set['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
train_set['num_valence_electrons'] = train_set['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
train_set['num_heteroatoms'] = train_set['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [118]:
test_set['mol'] = test_set['SMILES'].apply(lambda x: Chem.MolFromSmiles(x)) 
test_set['mol'] = test_set['mol'].apply(lambda x: Chem.AddHs(x))
test_set['num_of_atoms'] = test_set['mol'].apply(lambda x: x.GetNumAtoms())
test_set['num_of_heavy_atoms'] = test_set['mol'].apply(lambda x: x.GetNumHeavyAtoms())

test_set['tpsa'] = test_set['mol'].apply(lambda x: Descriptors.TPSA(x))
test_set['mol_w'] = test_set['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
test_set['num_valence_electrons'] = test_set['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
test_set['num_heteroatoms'] = test_set['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [119]:
train_x_array = np.array(train_set[['num_of_atoms','num_of_heavy_atoms','tpsa','mol_w','num_valence_electrons','num_heteroatoms']])
#train_x = np.concatenate((train_x, train_x_array), axis = 1)

In [120]:
test_x_array = np.array(test_set[['num_of_atoms','num_of_heavy_atoms','tpsa','mol_w','num_valence_electrons','num_heteroatoms']])
#test_x = np.concatenate((test_x, test_x_array), axis = 1)

In [20]:
#np.array(test_g).shape, np.array(test_ex).shape

In [21]:
#np.array(train2D).reshape(len(train2D), -1).shape, np.array(test2D).reshape(len(test2D), -1).shape

In [17]:
ex_data_ = np.array(ex_data).reshape(len(ex_data), -1)
g_data_ = np.array(g_data).reshape(len(g_data), -1)
#train2D_rs = np.array(train2D_).reshape(len(train2D_), -1)
#train_x = train2D_rs

train_x = np.concatenate((g_data, ex_data), axis = 1)
#train_x = np.concatenate((train_x, train2D_rs), axis = 1)
#train_x = np.concatenate((train_x, train_x_array), axis = 1)

#train_x = g_data_ - ex_data_
train_y = np.array(train_set.loc[:, ["Reorg_g", "Reorg_ex"]])

In [18]:
test_g_ = np.array(test_g).reshape(len(test_g), -1)
test_ex_ = np.array(test_ex).reshape(len(test_ex), -1)
#test2D_rs = np.array(test2D_).reshape(len(test2D_), -1)
#test_x = test2D_rs

test_x = np.concatenate((test_g, test_ex), axis = 1)
#test_x = np.concatenate((test_x, test2D_rs), axis = 1)
#test_x = np.concatenate((test_x, test_x_array), axis = 1)
#test_x = test_g_ - test_ex_

In [19]:
def rmse(y_true, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.sqrt(mse)

In [20]:
train_x.shape, test_x.shape, train_y.shape

((18157, 620, 16), (457, 620, 16), (18157, 2))

In [21]:
train_x[0].shape[0], train_x[0].shape[1]

(620, 16)

In [22]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001,amsgrad=True)
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,
#                                     beta_1=0.9,
#                                     beta_2=0.999,
#                                     epsilon=1e-07)

In [22]:
def build_model(input_layer):
    
    node = 64
    activation = 'relu'
    drop_rate = 0.2
    
    x = tf.keras.layers.Conv1D(node*4, 5, activation=activation)(input_layer)
    x = tf.keras.layers.MaxPooling1D(3)(x)
    x = tf.keras.layers.Conv1D(node*3, 5, activation=activation)(x)
    x = tf.keras.layers.MaxPooling1D(3)(x)
    x = tf.keras.layers.Conv1D(node, 5, activation=activation)(x)
    x = tf.keras.layers.MaxPooling1D(3)(x)
    x = tf.keras.layers.Flatten()(x)
        
    net = tf.keras.layers.Dense(2)(x)
    
    output_layer = tf.keras.models.Model(input_layer, net, name = 'build_model')

    return output_layer

In [23]:
def transformer_block(x,node,drop_rate,activation):
    attn_output = keras.layers.MultiHeadAttention(num_heads=2, 
                                        key_dim=node)(x, x)
    attn_output = keras.layers.Dropout(drop_rate)(attn_output)
    out1 = keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
    ffn_output = keras.layers.Dense(node, activation=activation)(out1) #
    ffn_output = keras.layers.Dense(node)(ffn_output) #
    ffn_output = keras.layers.Dropout(drop_rate)(ffn_output)
    return keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

def build_model(input_layer):
    
    node = 64
    activation = 'relu'
    drop_rate = 0.2
    avg_list = []
    inputs_list = []
    CuDNNlstm=True
        
    x = tf.keras.layers.Conv1D(node * 4, 3, activation=activation)(input_layer)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Conv1D(node * 3, 3, activation=activation)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.Conv1D(node, 3, activation=activation)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)

    #positions = tf.range(start=0, limit=x.shape[1], delta=1,dtype="float32") 
    #positions = tf.keras.layers.Embedding(input_dim=x.shape[1], output_dim=node)(positions)
    #x = x + positions
    
    #x = transformer_block(x,node,drop_rate,activation)
    #x = tf.keras.layers.GlobalMaxPooling1D()(x)
    #x = tf.keras.layers.SpatialDropout1D(drop_rate)(x)
    
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(x) # 100 .4
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(gru) # 100 .4
    gru = tf.keras.layers.SpatialDropout1D(drop_rate)(gru)
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(gru) # 100 .4
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(gru) # 100 .4
    gru = tf.keras.layers.SpatialDropout1D(drop_rate)(gru)
    gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = False))(gru) # 100 .4

    
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(x) # 100 .4
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(lstm) # 100 .4
    lstm = tf.keras.layers.SpatialDropout1D(drop_rate)(lstm)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(lstm) # 100 .4
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = True))(lstm) # 100 .4
    lstm = tf.keras.layers.SpatialDropout1D(drop_rate)(lstm)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, dropout=0.2, return_sequences = False))(lstm) # 100 .4
    
    x = tf.keras.layers.concatenate([gru, lstm])
    
    #gru = Bidirectional(CuDNNGRU(16, return_sequences = True))(x) # Bidirectional()
    #gru = TimeDistributed(tf.keras.layers.Dense(2))(gru)
    
    #lstm = Bidirectional(CuDNNLSTM(16, return_sequences = True))(x) # Bidirectional()
    #lstm = TimeDistributed(tf.keras.layers.Dense(2))(lstm)

    #x = tf.keras.layers.concatenate([gru, lstm])
    
    net = tf.keras.layers.Dense(2, activation = 'linear')(x)
    
    output_layer = tf.keras.models.Model(input_layer, net, name = 'build_model')

    return output_layer

In [45]:
def transformer_block(x,node,drop_rate,activation):
    attn_output = keras.layers.MultiHeadAttention(num_heads=2, 
                                        key_dim=node)(x, x)
    attn_output = keras.layers.Dropout(drop_rate)(attn_output)
    out1 = keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
    ffn_output = keras.layers.Dense(node, activation=activation)(out1) #
    ffn_output = keras.layers.Dense(node)(ffn_output) #
    ffn_output = keras.layers.Dropout(drop_rate)(ffn_output)
    return keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

def build_model(input_layer):
    
    node = 16
    activation = 'relu'
    drop_rate = 0.2
    avg_list = []
    inputs_list = []

    x = tf.keras.layers.Conv1D(node, 5, activation=activation)(input_layer)
    x = tf.keras.layers.MaxPooling1D(3)(x)
    x = tf.keras.layers.Dropout(drop_rate)(x)

    positions = tf.range(start=0, limit=x.shape[1], delta=1,dtype="float32") 
    positions = tf.keras.layers.Embedding(input_dim=x.shape[1], output_dim=node)(positions)
    x = x + positions
        
    x = transformer_block(x,node,drop_rate,activation)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.Dropout(drop_rate)(x)
    
    net = tf.keras.layers.Dense(2, activation='linear')(x)
    
    output_layer = tf.keras.models.Model(input_layer, net, name = 'build_model')

    return output_layer

In [28]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

2022-09-15 03:34:37.437012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 03:34:37.438039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 03:34:37.438692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 03:34:37.439372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 03:34:37.439982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

True

In [24]:
from keras.layers import Input, Dense
from keras.layers.wrappers import Bidirectional
from keras.layers import CuDNNLSTM, TimeDistributed, CuDNNGRU
from tensorflow import keras

In [38]:
input_dim = (train_x[0].shape[0], train_x[0].shape[1])
input_layer = Input(shape = input_dim)
model = build_model(input_layer)
model.compile(optimizer = optimizer, loss = rmse) # tf.keras.losses.MeanSquaredError()
model.fit(train_x, train_y, batch_size = 32, epochs = 20)
prediction = model.predict(test_x)

2022-09-15 17:43:38.095352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 17:43:38.096663: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 17:43:38.097358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 17:43:38.098878: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/20


2022-09-15 17:44:13.289756: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


568/568 [==============================] - 73s 73ms/step - loss: 0.3083
Epoch 2/20
568/568 [==============================] - 42s 73ms/step - loss: 0.3010
Epoch 3/20
568/568 [==============================] - 42s 74ms/step - loss: 0.2991
Epoch 4/20
568/568 [==============================] - 42s 74ms/step - loss: 0.2973
Epoch 5/20
568/568 [==============================] - 42s 74ms/step - loss: 0.2959
Epoch 6/20
568/568 [==============================] - 41s 73ms/step - loss: 0.2946
Epoch 7/20
568/568 [==============================] - 41s 73ms/step - loss: 0.2927
Epoch 8/20
568/568 [==============================] - 41s 73ms/step - loss: 0.2909
Epoch 9/20
568/568 [==============================] - 41s 73ms/step - loss: 0.2892
Epoch 10/20
568/568 [==============================] - 42s 74ms/step - loss: 0.2881
Epoch 11/20
568/568 [==============================] - 42s 73ms/step - loss: 0.2878
Epoch 12/20
568/568 [==============================] - 42s 73ms/step - loss: 0.2858
Epoch 13/20


KeyboardInterrupt: 

swish - loss 0.2507

In [ ]:
fold_num = 1
MAE_score = []
history_loss = []
train_loss = []
predictions = []
epochs = 25
kf = KFold(n_splits = 5, shuffle = True, random_state = SEED)
#input_dim = (train_x[0].shape[0], train_x[0].shape[1])
#input_layer = tf.keras.layers.Input(shape = input_dim)

for train_idx, valid_idx in kf.split(train_x):

    print(f"------------------fold{fold_num}----------------------")

    X_train, X_val = train_x[train_idx], train_x[valid_idx]
    y_train, y_val = train_y[train_idx], train_y[valid_idx]
    
    #print(X_train.shape, X_val.shape)
    print(y_train.shape, y_val.shape)

    input_dim = (X_train[0].shape[0], X_train[0].shape[1])
    input_layer = Input(shape = input_dim)
    
    model = build_model(input_layer)
    #model = my_dnn_model()

    model.compile(optimizer = optimizer, loss = rmse) # tf.keras.losses.MeanSquaredError()

    #earlystopper = tf.keras.callbacks.EarlyStopping(patience=10)
    earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

    history = model.fit(X_train, y_train, validation_data=(X_val,y_val),batch_size=32,epochs=epochs) #,callbacks,scheduler callbacks=[earlystopper]
    #history_loss.append(history.history['val_loss'])
    #train_loss.append(history.history['loss'])
    prediction = model.predict(test_x)
    val_prediction = model.predict(X_val)
    print(samsungrmse(val_prediction, y_val))

    predictions.append(prediction)

    fold_num += 1

------------------fold1----------------------
(14525, 2) (3632, 2)


2022-09-16 03:58:03.821600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 03:58:03.822847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 03:58:03.823633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 03:58:03.824633: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50


2022-09-16 03:58:39.902793: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


454/454 [==============================] - ETA: 0s - loss: 0.3113

2022-09-16 03:59:21.972336: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 144117760 exceeds 10% of free system memory.
2022-09-16 03:59:22.161139: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 144117760 exceeds 10% of free system memory.


454/454 [==============================] - 75s 97ms/step - loss: 0.3113 - val_loss: 0.3007
Epoch 2/50
454/454 [==============================] - 38s 83ms/step - loss: 0.3030 - val_loss: 0.2958
Epoch 3/50
454/454 [==============================] - 37s 83ms/step - loss: 0.3000 - val_loss: 0.2943
Epoch 4/50
454/454 [==============================] - 38s 83ms/step - loss: 0.2980 - val_loss: 0.3025
Epoch 5/50
454/454 [==============================] - 38s 84ms/step - loss: 0.2960 - val_loss: 0.2899
Epoch 6/50
454/454 [==============================] - 38s 83ms/step - loss: 0.2938 - val_loss: 0.2892
Epoch 7/50
454/454 [==============================] - 38s 84ms/step - loss: 0.2909 - val_loss: 0.2868
Epoch 8/50
454/454 [==============================] - 38s 84ms/step - loss: 0.2868 - val_loss: 0.2837
Epoch 9/50
454/454 [==============================] - 38s 83ms/step - loss: 0.2832 - val_loss: 0.2808
Epoch 10/50
454/454 [==============================] - 38s 83ms/step - loss: 0.2793 - val_los

In [25]:
# def rmse(y_pred, y_true):
#     return np.sqrt(((y_pred - y_true) ** 2).mean()) * 100

'''def samsungrmse(y_pred, y_true):
    all_rmse = []
    for idx in range(2):
        rmse = mean_squared_error(y_true[:, idx], y_pred[:, idx])
        rmse = (rmse/np.mean(np.abs(y_true[:, idx]))) * 100
        all_rmse.append(rmse)
    score = ((np.sum(all_rmse[0]) + np.sum(all_rmse[1])) / 2)
    return score'''

def samsungrmse(y_pred, y_true):
    all_rmse = []
    for idx in range(2):
        rmse = y_true[:,idx] - y_pred[:,idx]
        rmse = rmse ** 2
        rmse = rmse.mean()
        rmse = np.sqrt(rmse) * 100
        all_rmse.append(rmse)
    score = (np.sum(all_rmse[0]) + np.sum(all_rmse[1])) / 2
    return score

In [129]:
def single_samsungrmse(y_pred, y_true):
    rmse = y_true- y_pred
    rmse = rmse ** 2
    rmse = rmse.mean()
    rmse = np.sqrt(rmse) * 100
    score = rmse
    return score

In [130]:
from sklearn.model_selection import GroupKFold, RepeatedKFold
from sklearn.linear_model import BayesianRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [131]:
from sklearn.linear_model import LogisticRegression

In [132]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_tabnet.tab_model import TabNetRegressor

In [133]:
n_splits = 10
samsung_predictions = []
samsung_rmses = []
kfold = KFold(n_splits = n_splits, random_state = SEED, shuffle = True)
for i, (train_idx, val_idx) in enumerate(kfold.split(train_x)):
    val_predictions = []
    samsung_predictions_ = []
    y_vals = []
    for j in range(2):
        X_train, y_train = train_x[train_idx], train_y[train_idx, j]
        X_val, y_val = train_x[val_idx], train_y[val_idx, j]
        
        model = CatBoostRegressor(random_state=SEED, iterations=10000, task_type="GPU")
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0, early_stopping_rounds = 100)
        
        val_prediction = model.predict(X_val)
        prediction = model.predict(test_x)
        val_predictions.append(val_prediction)
        samsung_predictions_.append(prediction)
        y_vals.append(y_val)
        
        single_rmse = rmse(val_prediction, y_val)
        print(f'Fold {i} / single_rmse_{j} : {single_rmse}')
    
    samsung_rmse = samsungrmse(np.array(val_predictions).T, np.array(y_vals).T)
    samsung_rmses.append(samsung_rmse)
    print(f'Fold {i} /samsung_rmse : {samsung_rmse}')
    samsung_predictions.append(samsung_predictions_)

Fold 0 / single_rmse_0 : 33.2378368567877
Fold 0 / single_rmse_1 : 31.371414779743752
Fold 0 /samsung_rmse : 32.30462581826573
Fold 1 / single_rmse_0 : 32.75967164773025
Fold 1 / single_rmse_1 : 31.105396146416105
Fold 1 /samsung_rmse : 31.932533897073178
Fold 2 / single_rmse_0 : 32.961155346643054
Fold 2 / single_rmse_1 : 31.837035883179094
Fold 2 /samsung_rmse : 32.399095614911076
Fold 3 / single_rmse_0 : 32.633039262685614
Fold 3 / single_rmse_1 : 31.010431319032932
Fold 3 /samsung_rmse : 31.821735290859273
Fold 4 / single_rmse_0 : 32.75898893913298
Fold 4 / single_rmse_1 : 31.21710155788982
Fold 4 /samsung_rmse : 31.9880452485114
Fold 5 / single_rmse_0 : 32.70184019012935
Fold 5 / single_rmse_1 : 31.223603806425892
Fold 5 /samsung_rmse : 31.962721998277623
Fold 6 / single_rmse_0 : 33.40971427638996
Fold 6 / single_rmse_1 : 31.94346121533364
Fold 6 /samsung_rmse : 32.6765877458618
Fold 7 / single_rmse_0 : 31.970026445730397
Fold 7 / single_rmse_1 : 30.904505025468342
Fold 7 /samsung

In [ ]:
np.mean(samsung_rmses)

In [28]:
samsung_final_prediction = np.mean(np.array(predictions), axis=0)

NameError: name 'predictions' is not defined

In [135]:
samsung_final_prediction = np.mean(np.array(samsung_predictions), axis=0).T

In [ ]:
samsung_final_prediction

In [ ]:
for idx, col in enumerate(sample_submission.columns):
    if col=='index':
        continue
    sample_submission[col] = prediction[:,idx-1]
print('Done.')

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('final.csv', index = False)